# Gradienty w TensorFlow

W tej sekcji omówimy, jak w TensorFlow działa automatyczne różniczkowanie przy użyciu `tf.GradientTape`. Skupimy się na różnicach względem PyTorch:
* TensorFlow domyślnie działa w trybie eager (w TF 2.x), więc operacje i gradienty można śledzić „na żywo”.
* W TensorFlow obiekty `tf.Variable` domyślnie akumulują gradienty; zwykłe tensory wymagają jawnego śledzenia.
* Zamiast wywoływać `tensor.backward()`, używamy kontekstu `tf.GradientTape()` i metody `tape.gradient()`.


## Propagacja wsteczna w jednym kroku
Zaczniemy od zastosowania pojedynczej funkcji wielomianowej $y = f(x)$ do tensora $x$. Następnie wykonamy propagację wsteczną i wypiszemy gradient $\frac {dy} {dx}$.

$\begin{split}Function:\quad y &= 2x^4 + x^3 + 3x^2 + 5x + 1 \\
Derivative:\quad y' &= 8x^3 + 3x^2 + 6x + 5\end{split}$


## Wykonaj standardowe importy


In [4]:
import tensorflow as tf

## 2. Prosty przykład z `tf.Variable`

Tak jak w PyTorch używamy `requires_grad=True`, tak w TensorFlow tworzymy `tf.Variable`. Gradienty będą automatycznie śledzone dla operacji wykonanych wewnątrz `tf.GradientTape`.


In [12]:
f = lambda x: 2*x**4 + x**3 + 3*x**2 + 5*x + 1

In [13]:

x = tf.Variable(2.0)

with tf.GradientTape() as tape:
    y = f(x)

dy_dx = tape.gradient(y, x)
print('Wartość funkcji y:', y.numpy())
print('Gradient dy/dx:', dy_dx.numpy())


Wartość funkcji y: 63.0
Gradient dy/dx: 93.0


### Uwaga

Jeśli użyjemy `tf.constant` zamiast `tf.Variable`, trzeba jawnie poprosić o śledzenie:

In [14]:
x = tf.constant(2.0)
with tf.GradientTape() as tape:
    tape.watch(x)  # to dodajemy
    y = f(x)
dy_dx = tape.gradient(y, x)
print('Wartość funkcji y:', y.numpy())
print('Gradient dy/dx:', dy_dx.numpy())

Wartość funkcji y: 63.0
Gradient dy/dx: 93.0



W PyTorch wystarczy ustawić `requires_grad=True` podczas tworzenia tensora.

## 3. Różniczkowanie wieloetapowe

Sprawdźmy działanie `GradientTape` dla dwóch warstw przekształceń. Zauważ, że `GradientTape` śledzi wszystkie operacje w swojej przestrzeni kontekstu.


In [15]:
x = tf.Variable([[1., 2., 3.], [4., 5., 6.]])

with tf.GradientTape() as tape:
    tape.watch(x)
    y = tf.square(x)
    z = tf.reduce_sum(y)

# Gradient z = sum(x^2) względem x to 2*x
dz_dx = tape.gradient(z, x)
print('Gradient z względem x:', dz_dx.numpy())


Gradient z względem x: [[ 2.  4.  6.]
 [ 8. 10. 12.]]


Analogiczna operacja w PyTorch wyglądałaby następująco:

In [16]:
import torch

# zmienna wejściowa
x = torch.tensor([[1., 2., 3.],
                  [4., 5., 6.]], requires_grad=True)

# obliczenia
y = x ** 2
z = y.sum()

# backpropagation
z.backward()

# gradient z względem x
print("Gradient z względem x:", x.grad)


Gradient z względem x: tensor([[ 2.,  4.,  6.],
        [ 8., 10., 12.]])


## 4. Akumulacja gradientów vs resetowanie

W PyTorch gradienty w `.grad` domyślnie się kumulują, więc trzeba wywoływać `optimizer.zero_grad()`.

W TensorFlow po wywołaniu `tape.gradient()` gradienty nie są przechowywane – trzeba je zastosować i ewentualnie ponownie otworzyć kontekst.

TensorFlow udostępnia klasę optymalizatora, która przyjmuje gradienty wprost.


In [21]:
f(0)

25

In [23]:
# Przykład: krótkie trenowanie pojedynczego parametru
x = tf.Variable(3.0)
optimizer = tf.keras.optimizers.SGD(learning_rate=0.1)

for step in range(5):
    with tf.GradientTape() as tape:
        y = f(x)
    grad = tape.gradient(y, x)
    optimizer.apply_gradients([(grad, x)])
    print(f'Krok {step+1}: x = {x.numpy():.4f}, y = {y.numpy():.4f}, grad = {grad.numpy():.4f}')


Krok 1: x = 3.4000, y = 4.0000, grad = -4.0000
Krok 2: x = 3.7200, y = 2.5600, grad = -3.2000
Krok 3: x = 3.9760, y = 1.6384, grad = -2.5600
Krok 4: x = 4.1808, y = 1.0486, grad = -2.0480
Krok 5: x = 4.3446, y = 0.6711, grad = -1.6384


Analogiczny kod w PyTorch:

In [22]:
import torch

# parametr, który będziemy trenować
x = torch.tensor(3.0, requires_grad=True)

# optymalizator
optimizer = torch.optim.SGD([x], lr=0.1)

# pętla trenowania
for step in range(5):
    optimizer.zero_grad()     # wyczyść gradienty
    y = f(x)                  # oblicz stratę
    y.backward()              # policz gradient
    optimizer.step()          # zaktualizuj parametr

    print(f"Krok {step+1}: x = {x.item():.4f}, y = {y.item():.4f}, grad = {x.grad.item():.4f}")


Krok 1: x = 3.4000, y = 4.0000, grad = -4.0000
Krok 2: x = 3.7200, y = 2.5600, grad = -3.2000
Krok 3: x = 3.9760, y = 1.6384, grad = -2.5600
Krok 4: x = 4.1808, y = 1.0486, grad = -2.0480
Krok 5: x = 4.3446, y = 0.6711, grad = -1.6384


### Różnice względem TensorFlow:

* `y.backward()` ↔ `tape.gradient(...)`
* `optimizer.step()` ↔ `optimizer.apply_gradients(...)`
* w PyTorch trzeba **ręcznie wyzerować gradienty** (`optimizer.zero_grad()`), bo domyślnie są akumulowane.



## Drugi przykład – suma kilku kroków

Analogicznie do dynamicznego grafu w PyTorch możemy wykonywać wiele operacji, a gradienty obliczają się względem ostatniej funkcji celu.


In [26]:
x = tf.Variable(tf.ones((2, 3)))

with tf.GradientTape() as tape:
    tape.watch(x)
    y1 = 3 * x + 2
    y2 = tf.square(y1)
    y3 = tf.reduce_mean(y2)

dy3_dx = tape.gradient(y3, x)
print('y3 =', y3.numpy())
print('dy3/dx:', dy3_dx.numpy())


y3 = 25.0
dy3/dx: [[5. 5. 5.]
 [5. 5. 5.]]


W PyTorch

In [27]:
import torch

# zmienna wejściowa
x = torch.ones((2, 3), requires_grad=True)

# obliczenia
y1 = 3 * x + 2
y2 = y1 ** 2
y3 = y2.mean()

# backpropagation
y3.backward()

# gradient względem x
print("y3 =", y3.item())
print("dy3/dx:", x.grad)


y3 = 25.0
dy3/dx: tensor([[5., 5., 5.],
        [5., 5., 5.]])


🔎 Skąd ten gradient?

* $ y_1 = 3x + 2 $
* $ y_2 = (3x+2)^2 $
* $ y_3 = \text{mean}(y_2) $

$$
\frac{\partial y_3}{\partial x} = \frac{1}{N} \cdot 2(3x+2) \cdot 3 = \frac{6(3x+2)}{N}
$$

Dla
$$
x=1 \rightarrow 3x+2=5; N=6
$$
dlatego
$$
gradient = \frac{6\cdot 5}{6} = 5
$$


## 6. Gradienty względem wielu zmiennych

Możemy przekazać listę zmiennych. TensorFlow zwróci listę gradientów w tej samej kolejności. Poniżej prosty przykład dwuparametrowej funkcji.


In [ ]:
x = tf.Variable(1.0)
y = tf.Variable(2.0)

with tf.GradientTape() as tape:
    f_val = x**2 + x*y + y**2

grads = tape.gradient(f_val, [x, y])
print('df/dx =', grads[0].numpy())
print('df/dy =', grads[1].numpy())


## 7. Gradienty drugiego rzędu
    Aby obliczyć pochodne wyższego rzędu, trzeba użyć `persistent=True` w `GradientTape` lub zagnieżdżonych kontekstów.


In [ ]:
x = tf.Variable(2.0)

with tf.GradientTape(persistent=True) as tape2:
    with tf.GradientTape() as tape1:
        y = f(x)
    dy_dx = tape1.gradient(y, x)
d2y_dx2 = tape2.gradient(dy_dx, x)
print('dy/dx =', dy_dx.numpy())
print('d2y/dx2 =', d2y_dx2.numpy())


## 8. Porównanie z PyTorch

| Aspekt | PyTorch | TensorFlow |
| --- | --- | --- |
| Śledzenie gradientów | `requires_grad=True` | kontekst `tf.GradientTape`, `tf.Variable` śledzi domyślnie |
| Wyzwalanie backprop | `loss.backward()` | `tape.gradient(loss, vars)` |
| Kumulacja gradientów | `.grad` kumuluje, trzeba zerować | gradienty zwracane jednorazowo |
| Operacje in-place | Dozwolone (trzeba uważać) | Brak operacji in-place |
| Domyślne obliczenia | Tryb eager | Tryb eager (TF 2.x) |

    W praktyce schemat pracy jest podobny: definiujemy funkcję celu, obliczamy gradienty względem parametrów i aktualizujemy wagi przez optymalizator.


### Świetna robota!
